In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from dataset import L2ArcticDataset
from model import Model
from speechbrain.utils.edit_distance import wer_details_for_batch, alignment, op_table

In [15]:
ids = [["utt1"], ["utt2"]]
refs = [[["a", "b", "c", "d", "d"]], [["d", "e"]]]
hyps = [[["a", "b", "d", "c", "d"]], [["d", "e"]]]
wer_details = []
for ids_batch, refs_batch, hyps_batch in zip(ids, refs, hyps):
    details = wer_details_for_batch(
        ids_batch, refs_batch, hyps_batch, compute_alignments=True
    )
    wer_details.extend(details)
print(wer_details[0]["alignment"])
print(wer_details[1]["alignment"])

[('=', 0, 0), ('=', 1, 1), ('I', None, 2), ('=', 2, 3), ('=', 3, 4), ('D', 4, None)]
[('=', 0, 0), ('=', 1, 1)]


In [14]:
data = L2ArcticDataset(
    "train",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0", 
    "./data"
)

Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0272.TextGrid
Skipping malformatted TextGrid file: /home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0/YDCK/annotation/arctic_a0209.TextGrid


In [15]:
data[3]

(tensor([-6.8054e-03, -7.1716e-03, -6.6833e-03,  ...,  7.6294e-04,
          3.0518e-05,  1.2207e-04]),
 tensor([ 0, 12, 35, 12, 42,  0, 23,  9, 39, 45,  1, 48, 40,  3, 34, 43, 12, 36,
         23,  2, 42,  0, 45,  4, 48,  4, 33, 34, 15, 40, 42,  0, 42, 39,  2, 31,
          3, 22, 14,  0,  0]),
 tensor([ 0,  9, 35,  9, 42,  0, 48,  9, 39, 45,  1, 48, 40,  3, 34, 43, 12, 36,
         48,  2, 42,  0, 45,  4, 48,  4, 33, 34, 15, 40, 42,  0, 42, 39,  2, 31,
          3, 22, 14,  0,  0]),
 41,
 41)

In [16]:
import torch
import numpy as np
list(torch.utils.data.Subset(data, np.arange(1, 4)))

[(tensor([0.0179, 0.0180, 0.0183,  ..., 0.0058, 0.0063, 0.0069]),
  tensor([30, 14, 45, 11, 22, 12, 22, 12, 35, 42, 18, 23,  3,  9, 31,  0,  3, 44,
          23,  3, 45,  4, 42, 10,  0,  3, 35, 22, 20, 12, 29,  2, 35,  0, 40, 32,
          39,  3, 20, 12, 36,  0, 30, 12, 34, 40,  9, 33, 28,  0]),
  tensor([30, 14, 45, 11, 22, 12, 22, 12, 35, 42, 18, 23,  3,  9, 31,  0,  3, 44,
          23,  3, 45, 15, 42, 10,  0,  3, 35, 22, 20, 12, 29,  2, 35,  0, 40, 32,
          39,  3, 20, 12, 36, 32, 30, 12, 34, 40,  9, 33, 28,  0]),
  50,
  50),
 (tensor([0.0110, 0.0099, 0.0096,  ..., 0.0204, 0.0207, 0.0198]),
  tensor([47, 18, 34,  3, 40, 42,  0, 40, 33, 14, 37,  0, 30, 14, 10, 31, 22,  0,
           0]),
  tensor([47, 18, 34,  3, 40,  0,  3, 40, 33, 14, 37,  0, 30, 14, 10, 49, 22,  0,
           0]),
  19,
  19),
 (tensor([-6.8054e-03, -7.1716e-03, -6.6833e-03,  ...,  7.6294e-04,
           3.0518e-05,  1.2207e-04]),
  tensor([ 0, 12, 35, 12, 42,  0, 23,  9, 39, 45,  1, 48, 40,  3, 34, 43, 12

In [17]:
loader = torch.utils.data.DataLoader(
    data,
    batch_size=4,  # for bucketing
    shuffle=True,
    num_workers=4,
    drop_last=False,
    pin_memory=True,
    collate_fn=data.collate_fn,
)

In [18]:
for item in loader:
    display(item)
    break

[[tensor([ 0.0046,  0.0045,  0.0042,  ..., -0.0015, -0.0016, -0.0017]),
  tensor([-0.0011, -0.0011, -0.0015,  ...,  0.0027,  0.0028,  0.0031]),
  tensor([-0.0042, -0.0040, -0.0038,  ...,  0.0081,  0.0080,  0.0083]),
  tensor([0.0093, 0.0096, 0.0103,  ..., 0.0249, 0.0245, 0.0255])],
 tensor([[ 0,  8, 32, 11, 34, 28,  4, 39, 12, 35, 28,  4, 39, 34, 11, 41,  3, 35,
           0, 34,  4, 39,  5, 42,  3, 44, 32, 47, 17, 39, 14,  1, 40,  3, 42, 14,
          23,  3, 35,  9, 35, 14, 43, 12, 36,  9, 33, 40,  0,  0],
         [ 0, 34, 14, 35, 45,  8, 33,  8, 33, 29, 15,  5, 42, 42,  3, 20, 39, 14,
          23,  0,  3, 40, 37,  9, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0, 23,  9, 35,  0,  3, 35, 22,  2, 42,  0,  0, 40,  3, 37, 10,  0, 30,
          14, 42, 39,  8, 22, 42,  3, 28,  2, 23,  3, 34, 30, 10,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [30, 14, 45,  9

In [19]:
from tqdm import tqdm

In [20]:
for batch_id, (wav_view, *others) in enumerate(tqdm(loader, dynamic_ncols=True, desc='train')):
    display(batch_id)
    display(wav_view)
    display(*others)
    break
    # print(*others)

train:   0%|          | 0/712 [00:00<?, ?it/s]

0

[tensor([-0.0051, -0.0047, -0.0052,  ..., -0.0145, -0.0143, -0.0143]),
 tensor([-0.0043, -0.0045, -0.0048,  ...,  0.0191,  0.0190,  0.0188]),
 tensor([-0.0018, -0.0009, -0.0022,  ...,  0.0029,  0.0038,  0.0030]),
 tensor([ 6.1035e-05, -1.5259e-04,  1.2207e-04,  ...,  2.0508e-02,
          2.0874e-02,  2.0935e-02])]

tensor([[ 0, 41, 14, 42,  0, 10, 35, 22,  0, 12, 35,  2, 42, 23, 14, 30, 15, 42,
          9, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 37, 15, 48, 47, 18, 40,  4, 34, 14,  2, 42, 45, 10, 32, 43,
         39, 18, 23,  3, 45, 12, 35, 22, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 30, 14, 30,  2, 22,  0, 37, 14, 33, 22,  4, 28,  0, 30, 12, 48, 41,
         10, 42,  2, 35, 22, 45,  4, 48, 45,  8, 33, 22, 33, 14,  0, 45, 11, 44,
         12, 36, 12, 42,  0,  0],
        [ 0, 41, 14, 42, 10, 35, 22,  0, 12, 35,  2, 42, 23, 14, 30, 15, 42,  9,
         33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0]])

tensor([[ 0, 41, 14, 42,  4, 10, 35, 22,  0, 12, 35,  2,  0, 23, 14, 30,  4, 42,
          9, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 20, 15, 40, 47, 18, 40,  4, 34, 14,  2, 42, 44,  3, 32, 42,
         39, 18, 22,  3, 45, 12, 35, 22, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0],
        [ 0, 30, 14, 30,  9, 42,  0, 37, 14, 18,  0,  4, 28,  0, 30, 12, 48, 41,
         17, 42,  9, 35,  0, 45,  4,  0, 45,  8,  0, 22, 33, 14, 29, 45, 11, 44,
         12, 35, 12, 42,  0,  0],
        [ 0, 41, 14, 42, 10, 35, 22,  0, 14, 35,  2, 42, 22, 14, 30, 15, 42,  9,
         33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0]])

tensor([21, 29, 42, 21])

tensor([21, 29, 42, 21])

train:   0%|          | 0/712 [00:00<?, ?it/s]


In [21]:
data[400]

(tensor([-0.0026, -0.0026, -0.0024,  ...,  0.0006,  0.0004,  0.0002]),
 tensor([30, 14, 45, 12, 33, 28,  1, 33, 15,  3, 40, 40, 18, 35,  0,  0]),
 tensor([30, 14, 45, 12, 33, 28,  1, 33,  4,  3, 40, 40, 18, 35,  0,  0]),
 16,
 16)

In [22]:
data[3:5]

TypeError: list indices must be integers or slices, not str

In [ ]:
l1_model = Model(
    input_dim=self.upstream_dim,
    output_class_num=self.dataset['train'].output_class_num
    **self.modelrc,